In [ ]:
import os
import itertools
from attrs import define
# from code.train import train
# from codes.train_dist import train
from codes.train import train
from codes.optimizers.base import Optimizer
# from code.problems import Problem
from codes import Loss
from codes.datasets import Dataset
from codes.models import Model
from codes.attacks import Attack

# %matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
def zip_dict(**kwargs):
    keys = kwargs.keys()
    for instance in zip(*kwargs.values()):
        yield dict(zip(keys, instance))


def product_dict(**kwargs):
    keys = kwargs.keys()
    for instance in itertools.product(*kwargs.values()):
        yield dict(zip(keys, instance))

In [ ]:
os.environ["MKL_THREADING_LAYER"] = "AMD"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["TORCH_DEVICE"] = "cpu"

In [ ]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_CHECK_EXIST'] = 'False'
# os.environ['MLFLOW_CHECK_EXIST'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

In [ ]:
@define
class BaseConfig():
    nepochs:         int = 5
    npeers:          int = 150
    seed:            int = 0

    loss:           Loss = Loss.MSELoss
    model:         Model = Model.Mean
    dataset:     Dataset = Dataset.Normal

    # nsamples:        int = 1000
    valenabled_:    bool = None
    valnsamples_:    int = None
    nclasses:        int = None
    hratio:        float = None

    optimizer: Optimizer = None
    batchsize:       int = 100
    lr:            float = 1e-2
    attack:       Attack = None

    trueweights:    bool = None

    mdbatchsize_:    int = None
    mdniters_:       int = None
    mdlr_:           int = None
    mdfull_:        bool = None

In [ ]:
os.environ['MLFLOW_RUN_TAGS'] = str(dict(about=f'final'))

args_grid = dict(
    hratio=[0.001, 0.01, 0.1],
    mdlr_=[3.5, 4.5, 12.5],
)

for d in zip_dict(**args_grid):

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.optimizer = Optimizer.SGD
    config.trueweights = False
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.hratio = None
    config.optimizer = Optimizer.SGD
    config.trueweights = True
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
    config = BaseConfig(**d)
    config.optimizer = Optimizer.MeritFed
    config.mdfull_ = True
    config.mdniters_ = 10
    config.mdbatchsize_ = 100
    config.valenabled_ = True
    config.valnsamples_ = 1000
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
    config = BaseConfig(**d)
    config.optimizer = Optimizer.MeritFed
    config.mdfull_ = False
    config.mdniters_ = 10
    config.mdbatchsize_ = 100
    config.valenabled_ = True
    config.valnsamples_ = 1000
    %time train(config)

    config = BaseConfig(**d)
    config.optimizer = Optimizer.TAWT
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    %time train(config)

    config = BaseConfig(**d)
    config.optimizer = Optimizer.FedAdp
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    %time train(config)

In [ ]:
os.environ['MLFLOW_RUN_TAGS'] = str(dict(about=f'FedAvg'))
args_grid = dict(
    hratio=[0.001, 0.01, 0.1],
)

for d in zip_dict(**args_grid):

    os.environ['MLFLOW_RUN_NAME'] = 'FedAvg K=10'  # one have to adjust K manually in single_thread.py
    config = BaseConfig(**d)
    config.optimizer = Optimizer.FedAvg
    %time train(config)
    # break

In [ ]:
@define
class BaseConfig():
    nepochs:         int = 2
    npeers:          int = 150
    seed:            int = 0

    loss:           Loss = Loss.MSELoss
    model:         Model = Model.Mean
    dataset:     Dataset = Dataset.Normal

    # nsamples:        int = 1000
    valenabled_:    bool = None
    valnsamples_:    int = None
    nclasses:        int = None
    hratio:        float = None

    optimizer: Optimizer = None
    batchsize:       int = 100
    lr:            float = 1e-2
    attack:       Attack = Attack.IPM # one have to change attack type here

    trueweights:    bool = None

    mdbatchsize_:    int = None
    mdniters_:       int = None
    mdlr_:           int = None
    mdfull_:        bool = None



In [ ]:
os.environ['MLFLOW_RUN_TAGS'] = str(dict(about=f'byz'))

args_grid = dict(
    hratio=[0.001, 0.01, 0.1],
    mdlr_=[3.5, 4.5, 12.5],
)

for d in zip_dict(**args_grid):

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.optimizer = Optimizer.SGD
    config.trueweights = False
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.hratio = None
    config.optimizer = Optimizer.SGD
    config.trueweights = True
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
    config = BaseConfig(**d)
    config.optimizer = Optimizer.MeritFed
    config.mdfull_ = True
    config.mdniters_ = 10
    config.mdbatchsize_ = 100
    config.valenabled_ = True
    config.valnsamples_ = 1000
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
    config = BaseConfig(**d)
    config.optimizer = Optimizer.MeritFed
    config.mdfull_ = False
    config.mdniters_ = 10
    config.mdlr_ = 10
    config.mdbatchsize_ = 100
    config.valenabled_ = True
    config.valnsamples_ = 1000
    %time train(config)

    config = BaseConfig(**d)
    config.optimizer = Optimizer.TAWT
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    %time train(config)

    config = BaseConfig(**d)
    config.optimizer = Optimizer.FedAdp
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    %time train(config)

#### 